In [5]:
!playwright install --with-deps chromium 

In [9]:
import logging
import sys
import json
import asyncio
import os

notebook_dir = os.getcwd()  

parent_dir = os.path.dirname(notebook_dir)

sys.path.append(parent_dir)
from autogen.agentchat.contrib.magentic_one.websurfer import MultimodalWebSurfer
from autogen.agentchat.contrib.magentic_one.coder_agent import create_coder_agent
from autogen.agentchat.contrib.magentic_one.filesurfer_agent import create_file_surfer
from autogen.agentchat.contrib.magentic_one.orchestrator_agent import OrchestratorAgent
from autogen.agentchat import AssistantAgent, UserProxyAgent, ConversableAgent
import tempfile
from autogen.coding import DockerCommandLineCodeExecutor


logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Add console handler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

# NOTE: Don't forget to 'playwright install --with-deps chromium'

# Set up logging configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler('orchestrator.log')
    ]
)

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


In [10]:
config_list = [
    {
        "model": os.environ.get("MODEL"),
        "api_key": os.environ.get("API_KEY"),
        "base_url": os.environ.get("BASE_URL", None)
    }
]

llm_config = {
    "config_list": config_list,
    "temperature": 0,
    "timeout": 120
}


temp_dir = tempfile.TemporaryDirectory()

In [12]:
executorENV = DockerCommandLineCodeExecutor(
    image="python:3.12-slim",  
    timeout=10,  
    work_dir=temp_dir.name, 
)

executor = ConversableAgent(
    name="executor",
    description="""A computer terminal that performs no other action than running Python scripts (provided to it quoted in ```python code blocks), or sh shell scripts (provided to it quoted in ```sh code blocks)""",
    llm_config=False,  
    code_execution_config={"executor": executorENV}, 
    human_input_mode="NEVER",  
)

In [13]:
# Create the web surfer agent
websurfer = MultimodalWebSurfer(
    name="WebSurfer",
    human_input_mode="TERMINATE",
    llm_config=llm_config
)

# Initialize the browser
await websurfer.init(
    headless=True,
    downloads_folder="./downloads",
    start_page="https://www.bing.com",
    browser_channel="chromium",
    to_save_screenshots=True
)

/Users/merlin.von-trott/Documents/merlin/contrib/ag2/autogen/agentchat/conversable_agent.py:2626: UserWarning: Function 'visit_url' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
/Users/merlin.von-trott/Documents/merlin/contrib/ag2/autogen/agentchat/conversable_agent.py:2626: UserWarning: Function 'web_search' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
/Users/merlin.von-trott/Documents/merlin/contrib/ag2/autogen/agentchat/conversable_agent.py:2626: UserWarning: Function 'history_back' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
/Users/merlin.von-trott/Documents/merlin/contrib/ag2/autogen/agentchat/conversable_agent.py:2626: UserWarning: Function 'page_up' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
/U

In [14]:
coder = create_coder_agent("Coder",llm_config=llm_config)
filesurfer = create_file_surfer("FileSurfer",llm_config=llm_config)

In [15]:
# choose the agents for the orchestrator
orchestrator = OrchestratorAgent(
    name="Orchestrator",
    llm_config=llm_config,
    agents=[websurfer],  #coder, filesurfer, executor, 
    max_consecutive_auto_reply=10,
    max_stalls_before_replan=3,  
    max_replans=3, 
    return_final_answer=True,
    description="An orchestrator that manages conversation flow and handles errors."
)

In [ ]:
task = """Find out some detailed informtion about the magentic one agent from microsoft"""
#task = """Go to amazon.com and find a good beginner 3d printer, pick any good one for under 300 dollars. """
#task = """write the game sname in python """
#task = """can you generate the python call to run all return all the prime numbers bewteen 1 and 1000 and return the prime numbers ? (give the code to the Executor to run it, don't ask the user to run it. If it does not work with the executor, tell the user.)"""

messages = [
    {"role": "user", "content": task}]

#result = await coder.a_generate_reply(messages)
#result = await executor.a_generate_reply(messages)

result = await orchestrator.a_generate_reply(messages=messages)

    

2024-12-17 18:40:55,048 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-17 18:40:56,875 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-17 18:40:56,885 - autogen.logger.file_logger - INFO - {"source_id": 4695657520, "source_name": "Orchestrator", "event_name": "thought", "json_state": "{\"data\": {\"stage\": \"initial_plan\", \"plan\": \"\\nWe are working to address the following user request:\\n\\nFind out some detailed informtion about the magentic one agent from microsoft\\n\\n\\nTo answer this request we have assembled the following team:\\n\\nWebSurfer: A helpful assistant with access to a web browser. Ask them to perform web searches, open pages, and interact with content (e.g., clicking links, scrolling the viewport, etc., filling in form fields, etc.) It can also summarize the entire page, or answer questions based on the content of the page. It can also be asked to

2024-12-17 18:41:12,094 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-17 18:41:21,208 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-17 18:41:40,434 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-17 18:41:59,233 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [ ]:
result